In [1]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install resampy

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten
from keras.models import Model
from tensorflow.keras.layers import GlobalMaxPool1D
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Conv1D, GlobalMaxPooling1D
from keras.models import Model
from keras.optimizers import Adam

In [4]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

def process_directory(directory):
    features = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(root, file)
                features.append(extract_features(file_path))
    return features

# Replace 'your_directory_path' with the path of your directory

Italian = "nyc\Italian"
Italian_features = process_directory(Italian)

Japanese = "nyc\Japanese"
Japanese_features = process_directory(Japanese)

Korean = "nyc\Korean"
Korean_features = process_directory(Korean)

Mandarin = "nyc\Mandarin"
Mandarin_features = process_directory(Mandarin)

Polish = "nyc\Polish"
Polish_features = process_directory(Polish)

Portuguese = "nyc\Portuguese"
Portuguese_features = process_directory(Portuguese)

Russian = "nyc\Russian"
Russian_features = process_directory(Russian)

Spanish = "nyc\Spanish"
Spanish_features = process_directory(Spanish)

for feature in features:
    print(Mandarin_features)

[array([-2.6207135e+02,  1.0340501e+02,  1.2674357e+01,  3.3375668e+01,
        1.6510799e+01,  5.2576804e+00, -5.9591269e+00,  5.6274190e+00,
       -1.3636049e+01,  5.0838131e-01, -7.0773964e+00, -1.8243937e+00,
       -9.3465862e+00,  3.5044758e+00, -5.9606380e+00, -4.5942497e+00,
       -4.3258624e+00,  1.1531072e+00, -5.6149998e+00, -6.8228847e-01,
       -2.1697345e+00, -3.7792137e+00, -4.4892654e+00,  1.6653459e-01,
       -1.0120521e+00,  2.1717737e+00, -1.1603183e+00,  3.8398769e+00,
        4.2559142e+00,  5.2030292e+00,  5.7429576e+00,  6.8019309e+00,
        8.1168795e+00,  4.7329926e+00,  2.8789244e+00,  7.7039129e-01,
        3.9473317e+00,  4.7978410e-03, -8.0938220e-01,  2.2605386e+00],
      dtype=float32), array([-2.6621951e+02,  8.3981781e+01,  7.1737885e+00,  4.1349934e+01,
        1.7894642e+01,  1.1237135e+01, -9.2850327e-01,  9.5411263e+00,
       -1.0720953e+01,  1.6707794e+01, -1.9913615e+00,  1.0360796e+01,
        3.4375346e+00,  8.9871855e+00, -2.8766441e+00

In [62]:
# Create a DataFrame for each accent
df_italian = pd.DataFrame()
df_italian['Vectors'] = Italian_features
df_italian['Accent'] = "Italian"

df_japanese = pd.DataFrame()
df_japanese['Vectors'] = Japanese_features
df_japanese['Accent'] = "Japanese"

df_korean = pd.DataFrame()
df_korean['Vectors'] = Korean_features
df_korean['Accent'] = "Korean"

df_mandarin = pd.DataFrame()
df_mandarin['Vectors'] = Mandarin_features
df_mandarin['Accent'] = "mandarin"

df_polish = pd.DataFrame()
df_polish['Vectors'] = Polish_features
df_polish['Accent'] = "polish"

df_portuguese = pd.DataFrame()
df_portuguese['Vectors'] = Portuguese_features
df_portuguese['Accent'] = "Portuguese"

df_russian = pd.DataFrame()
df_russian['Vectors'] = Russian_features
df_russian['Accent'] = "Russian"

df_spanish = pd.DataFrame()
df_spanish['Vectors'] = Spanish_features
df_spanish['Accent'] = "Spanish"

# Append all the DataFrames together
df = pd.concat([df_mandarin, df_polish, df_italian, df_japanese, df_korean, df_portuguese, df_russian, df_spanish], ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)

# Display the DataFrame
print(df)

                                               Vectors    Accent
0    [-262.07135, 103.40501, 12.674357, 33.375668, ...  mandarin
1    [-266.2195, 83.98178, 7.1737885, 41.349934, 17...  mandarin
2    [-324.1605, 104.032906, 3.6130328, 30.639978, ...  mandarin
3    [-296.7087, 107.27797, 9.575731, 40.041893, 16...  mandarin
4    [-265.4242, 105.13989, 0.55889285, 19.527306, ...  mandarin
..                                                 ...       ...
464  [-353.7568, 111.01047, -0.57015914, 12.17861, ...   Spanish
465  [-244.14275, 76.057076, -12.995524, 21.355549,...   Spanish
466  [-414.2467, 86.88756, -38.184223, 27.91933, -1...   Spanish
467  [-359.83334, 94.913795, 14.560818, 31.941372, ...   Spanish
468  [-339.36234, 131.82262, 4.1954703, 3.993738, -...   Spanish

[469 rows x 2 columns]


In [63]:
df = pd.read_csv("output.csv")

In [64]:
X_train, X_test, y_train, y_test = train_test_split(df['Vectors'], df['Accent'], test_size=0.2)

In [65]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [66]:
# Define the model
# inputs = Input(shape=(1,))  # replace 10 with the dimension of your input data
# outputs = Dense(1)(inputs)  # replace 1 with the dimension of your output data
# model = Model(inputs=inputs, outputs=outputs)
# from keras.layers import Input

# Assuming your model's current input layer is like this:
# input_layer = Input(shape=(10,))

# Modify the input shape to match your data
# input_layer = Input(shape=(375,))

# Then rebuild the rest of your model architecture using this modified input layer
# model = ...  # Rebuild your model with input_layer as the first layer


In [67]:
input_ids = Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(None,), dtype=tf.int32, name="attention_mask")

bert_output = bert_model(input_ids, attention_mask=attention_mask)
sequence_output = bert_output.last_hidden_state

cnn_layer = Conv1D(filters=64, kernel_size=5, activation='relu')(sequence_output)
cnn_layer = layers.Flatten()
cnn_layer = layers.Dense(10,activation='softmax')
cnn_output = cnn_layer(sequence_output)
cnn_output = GlobalMaxPool1D()(cnn_output)
lstm_output = LSTM(64, return_sequences=True)(sequence_output)
lstm_output = GlobalMaxPool1D()(lstm_output)
clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

In [68]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [69]:
X_train

360    [-3.4465994e+02  7.3764404e+01  1.0771570e+00 ...
34     [-2.24209366e+02  1.18454185e+02 -1.60102558e+...
192    [-2.4172882e+02  1.2840857e+02  3.2223919e+01 ...
123    [-3.4153204e+02  1.2129148e+02 -3.7617256e+01 ...
80     [-3.1817401e+02  1.3825887e+02 -2.5708876e+01 ...
                             ...                        
314    [-146.50687      40.59463     -10.745815     2...
367    [-3.0876270e+02  9.0595421e+01 -1.4628713e+00 ...
401    [-336.8476     113.170525     5.961355    31.3...
419    [-2.6795349e+02  1.0905945e+02  3.0386564e+01 ...
63     [-3.37711426e+02  1.08642555e+02 -3.15810037e+...
Name: Vectors, Length: 375, dtype: object

In [77]:
print(X_train.dtype)
print(y_train.dtype)
print(X_train.shape)
# X_train = X_train.astype(int)
X_train

object
object
(375,)


154    [-262.9696       91.63474      -8.957172     2...
19     [-2.6075272e+02  5.8539864e+01  3.0065472e+01 ...
349    [-3.1785767e+02  8.2508911e+01 -1.4683044e+01 ...
361    [-3.09203003e+02  6.53718109e+01  6.35384226e+...
53     [-3.0855386e+02  9.8762497e+01 -1.1973377e+01 ...
                             ...                        
228    [-3.2007095e+02  9.6162285e+01  1.3048472e+01 ...
122    [-2.9150269e+02  7.6513809e+01 -3.4338524e+01 ...
214    [-3.4276266e+02  1.0364148e+02 -5.3493538e+01 ...
251    [-4.2669226e+02  4.5991116e+01  1.5897790e+01 ...
467    [-3.5983334e+02  9.4913795e+01  1.4560818e+01 ...
Name: Vectors, Length: 375, dtype: object

In [ ]:
print(X_train.dtype)

object


In [ ]:
"""from sklearn.preprocessing import OneHotEncoder

# Assuming X_train is your feature data
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
X_train_encoded = encoder.fit_transform(X_train.values.reshape(-1, 1))"""

'from sklearn.preprocessing import OneHotEncoder\n\n# Assuming X_train is your feature data\nencoder = OneHotEncoder(sparse=False)\n\n# Fit and transform the data\nX_train_encoded = encoder.fit_transform(X_train.values.reshape(-1, 1))'

In [ ]:
history = model.fit(X_train, batch_size=32, epochs=3, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [135]:
from decimal import Decimal, getcontext

# Set the precision for Decimal calculations
getcontext().prec = 50  # Example: Set precision to 50 decimal places

# List of values in scientific notation (positive and negative)
values = [1.18454185e+02, -3.14159265e+00, 2.71828183e+00, -5.678e-03]

# Convert scientific notation values to Decimal array
decimal_values = [Decimal(value) for value in values]

# Create Decimal numbers with high precision
y = Decimal('9.87654321098765432109876543210987654321098765432109')

# Perform calculations with Decimal numbers
results = [x for x in decimal_values]

print(results)  # Output: [1170.241384822327037087991315115687107897611553590, -30.974035214200162719693304895547013793834907734, 26.706861666247433752535370207087283170317465037, -0.056063692590370294750741585512655555574342138556]


[Decimal('118.45418499999999539795680902898311614990234375'), Decimal('-3.141592650000000208621031561051495373249053955078125'), Decimal('2.718281830000000010016947271651588380336761474609375'), Decimal('-0.005677999999999999887478896454240384628064930438995361328125')]


In [136]:
from decimal import Decimal, getcontext

# Set the precision for Decimal calculations
getcontext().prec = 50  # Example: Set precision to 50 decimal places

# List of values in scientific notation (positive and negative)
values = [1.18454185e+02, -3.14159265e+00, 2.71828183e+00, -5.678e-03]

# Convert scientific notation values to Decimal array
decimal_values = [Decimal(value) for value in values]

# Perform calculations with Decimal numbers
results = [x for x in decimal_values]

print(results)  # Output: [1170.241384822327037087991315115687107897611553590, -30.974035214200162719693304895547013793834907734, 26.706861666247433752535370207087283170317465037, -0.056063692590370294750741585512655555574342138556]


[Decimal('118.45418499999999539795680902898311614990234375'), Decimal('-3.141592650000000208621031561051495373249053955078125'), Decimal('2.718281830000000010016947271651588380336761474609375'), Decimal('-0.005677999999999999887478896454240384628064930438995361328125')]


In [137]:
data = pd.read_csv("output.csv")

In [138]:
data["Vectors"]

0      [-2.6207135e+02  1.0340501e+02  1.2674357e+01 ...
1      [-2.6621951e+02  8.3981781e+01  7.1737885e+00 ...
2      [-3.24160492e+02  1.04032906e+02  3.61303282e+...
3      [-2.9670871e+02  1.0727797e+02  9.5757313e+00 ...
4      [-2.6542419e+02  1.0513989e+02  5.5889285e-01 ...
                             ...                        
464    [-3.5375681e+02  1.1101047e+02 -5.7015914e-01 ...
465    [-2.4414275e+02  7.6057076e+01 -1.2995524e+01 ...
466    [-4.1424670e+02  8.6887558e+01 -3.8184223e+01 ...
467    [-3.5983334e+02  9.4913795e+01  1.4560818e+01 ...
468    [-339.36234     131.82262       4.1954703     ...
Name: Vectors, Length: 469, dtype: object

In [141]:
for i in range (500):
    print(data[:])

                                               Vectors    Accent
0    [-2.6207135e+02  1.0340501e+02  1.2674357e+01 ...  mandarin
1    [-2.6621951e+02  8.3981781e+01  7.1737885e+00 ...  mandarin
2    [-3.24160492e+02  1.04032906e+02  3.61303282e+...  mandarin
3    [-2.9670871e+02  1.0727797e+02  9.5757313e+00 ...  mandarin
4    [-2.6542419e+02  1.0513989e+02  5.5889285e-01 ...  mandarin
..                                                 ...       ...
464  [-3.5375681e+02  1.1101047e+02 -5.7015914e-01 ...   Spanish
465  [-2.4414275e+02  7.6057076e+01 -1.2995524e+01 ...   Spanish
466  [-4.1424670e+02  8.6887558e+01 -3.8184223e+01 ...   Spanish
467  [-3.5983334e+02  9.4913795e+01  1.4560818e+01 ...   Spanish
468  [-339.36234     131.82262       4.1954703     ...   Spanish

[469 rows x 2 columns]
                                               Vectors    Accent
0    [-2.6207135e+02  1.0340501e+02  1.2674357e+01 ...  mandarin
1    [-2.6621951e+02  8.3981781e+01  7.1737885e+00 ...  mandarin
2

In [143]:
print(data[1:])

                                               Vectors    Accent
1    [-2.6621951e+02  8.3981781e+01  7.1737885e+00 ...  mandarin
2    [-3.24160492e+02  1.04032906e+02  3.61303282e+...  mandarin
3    [-2.9670871e+02  1.0727797e+02  9.5757313e+00 ...  mandarin
4    [-2.6542419e+02  1.0513989e+02  5.5889285e-01 ...  mandarin
5    [-3.2014514e+02  8.7206871e+01 -1.1844902e+01 ...  mandarin
..                                                 ...       ...
464  [-3.5375681e+02  1.1101047e+02 -5.7015914e-01 ...   Spanish
465  [-2.4414275e+02  7.6057076e+01 -1.2995524e+01 ...   Spanish
466  [-4.1424670e+02  8.6887558e+01 -3.8184223e+01 ...   Spanish
467  [-3.5983334e+02  9.4913795e+01  1.4560818e+01 ...   Spanish
468  [-339.36234     131.82262       4.1954703     ...   Spanish

[468 rows x 2 columns]
